In [32]:
import numpy as np
import pandas as pd
#匯入Yahoo!Finance
#pip install 
import yfinance as yf
import datetime as dt

In [33]:
yf.pdr_override()

#可以替換成其他貨幣
# https://finance.yahoo.com/quote/BTC-USD?p=BTC-USD&.tsrc=fin-srch

symbol = "BTC-USD"
start =dt.date(2018, 1, 1)
end = dt.date(2022, 12, 31)

# Read data
df = yf.download(symbol, start, end)
df_Indicators=yf.download(symbol, start, end)
df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-01,14112.200195,14112.200195,13154.700195,13657.200195,13657.200195,10291200000
2018-01-02,13625.000000,15444.599609,13163.599609,14982.099609,14982.099609,16846600192
2018-01-03,14978.200195,15572.799805,14844.500000,15201.000000,15201.000000,16871900160
2018-01-04,15270.700195,15739.700195,14522.200195,15599.200195,15599.200195,21783199744
2018-01-05,15477.200195,17705.199219,15202.799805,17429.500000,17429.500000,23840899072
...,...,...,...,...,...,...
2022-12-26,16842.250000,16920.123047,16812.369141,16919.804688,16919.804688,11886957804
2022-12-27,16919.291016,16959.845703,16642.072266,16717.173828,16717.173828,15748580239
2022-12-28,16716.400391,16768.169922,16497.556641,16552.572266,16552.572266,17005713920


In [34]:
def SMA(close,n):   
    return close.rolling(window=n).mean()

In [35]:
def WMA(close, n):
    weights = np.array(range(1, n+1))
    sum_weights = np.sum(weights)

    res = close.rolling(window=n).apply(lambda x: np.sum(weights*x) / sum_weights, raw=False)
    return res

In [36]:
def EMA(close, n):    
    res=close.ewm(span=n,min_periods=n).mean()
    return res

In [37]:
def RSI(close, n):
    # RSI
    change = close.diff(1)
    df["Gain"] = change.mask(change < 0, 0)
    df["Loss"] = abs(change.mask(change > 0, 0))
    df["AVG_Gain"] = df.Gain.rolling(n).mean()
    df["AVG_Loss"] = df.Loss.rolling(n).mean()
    df["RS"] = df["AVG_Gain"] / df["AVG_Loss"]
    df["RSI"] = 100 - (100 / (1 + df["RS"]))
    return df["RSI"]


In [38]:
def Stoch_RSI(close, n):
    #Stochastic RSI
    RSI=df["RSI"]
    LL_RSI = RSI.rolling(n).min()
    HH_RSI = RSI.rolling(n).max()         
    return (RSI - LL_RSI) / (HH_RSI - LL_RSI)

In [39]:
def BB(close, n):
    MA = pd.Series(close.rolling(n).mean())
    STD = pd.Series(close.rolling(n).std())
    bb1 = MA + 2 * STD
    df["Upper Bollinger Band"] = pd.Series(bb1)
    bb2 = MA - 2 * STD
    df["Lower Bollinger Band"] = pd.Series(bb2)
    return df["Upper Bollinger Band"],df["Lower Bollinger Band"]

In [40]:
def MACD(close):
    df['EMA12'] = close.ewm(span=12, adjust=False).mean() 
    df['EMA26'] = close.ewm(span=26, adjust= False).mean() 
    return df['EMA12'] - df['EMA26'] 


In [41]:
# 指標副程式(資料,天數)
df_Indicators['SMA_7']=SMA(df['Adj Close'],7)
df_Indicators['SMA_14']=SMA(df['Adj Close'],14)

df_Indicators['EMA_7']=EMA(df['Adj Close'],7)
df_Indicators['EMA_14']=EMA(df['Adj Close'],14)

df_Indicators['RSI']=RSI(df['Adj Close'],14)


df_Indicators['Stoch_RSI']=Stoch_RSI(df['Adj Close'],14)

df_Indicators['Upper_BB']=BB(df['Adj Close'],20)[0]
df_Indicators['Lower_BB']=BB(df['Adj Close'],20)[1]

df_Indicators['MACD']=MACD(df['Adj Close'])
df_Indicators= df_Indicators.dropna()
df_Indicators

,Open,High,Low,Close,Adj Close,Volume,SMA_7,SMA_14,EMA_7,EMA_14,RSI,Stoch_RSI,Upper_BB,Lower_BB,MACD
Date,,,,,,,,,,,,,,,
2018-01-28,11475.299805,12040.299805,11475.299805,11786.299805,11786.299805,8350360064,11259.571708,11635.535854,11516.047002,11884.539097,37.129744,0.560353,15192.188206,9606.291969,-761.452864
2018-01-29,11755.500000,11875.599609,11179.200195,11296.400391,11296.400391,7107359744,11311.714565,11455.293039,11461.122271,11804.864485,34.531168,0.835160,14866.144405,9602.435771,-744.090581
2018-01-30,11306.799805,11307.200195,10036.200195,10106.299805,10106.299805,8637859840,11202.843052,11356.421596,11122.356157,11575.251876,40.137091,1.000000,14450.867575,9531.012601,-816.944788
2018-01-31,10108.200195,10381.599609,9777.419922,10221.099609,10221.099609,8041160192,11040.228655,11287.314453,10897.011838,11392.534553,42.917380,1.000000,14318.165125,9345.245031,-855.556563
2018-02-01,10237.299805,10288.799805,8812.280273,9170.540039,9170.540039,9959400448,10741.820033,11122.717285,10465.350527,11093.196564,34.828472,0.651247,14132.400116,9050.004084,-959.863343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,16842.250000,16920.123047,16812.369141,16919.804688,16919.804688,11886957804,16851.525949,16968.767718,16862.360055,16901.268372,45.248727,0.118067,17703.255616,16318.430126,-96.481656
2022-12-27,16919.291016,16959.845703,16642.072266,16717.173828,16717.173828,15748580239,16824.507254,16892.757394,16826.063498,16876.722433,29.877219,0.000000,17705.606137,16302.984292,-103.142239
2022-12-28,16716.400391,16768.169922,16497.556641,16552.572266,16552.572266,17005713920,16786.655413,16802.537528,16757.690690,16833.502411,27.236901,0.000000,17690.567984,16249.932211,-120.315843


In [42]:
df_Indicators.to_csv('BTC-Indicators.csv')